In [0]:
"""
mount_lv426_blobstorage.py

Securely mounts Azure Blob Storage containers to Databricks using secrets stored in Azure Key Vault.

- Uses `dbutils.secrets.get` to retrieve the storage account key.
- Mounts each container only if not already mounted.
- Mount point format: /mnt/<container_name>

Required:
- Azure Key Vault-backed secret scope (e.g., "databricks-secrets-lv426")
- Secret key containing the storage key (e.g., "lv426-storage-key")

Typical Use:
Run this once during environment setup to mount:
    - raw-ingest
    - external-ingest
"""


Azure Key Vault Method

In [0]:
# Parameters
storage_account_name = "datalakelv426"
containers = ["raw-ingest", "external-ingest", "adf-silver"]  
mount_base = "/mnt"


# Azure Key Vault-backed secret scope setup
secret_scope = "databricks-secrets-lv426"  # Must match the scope name created via CLI
secret_key_name = "lv426-storage-key"      # Must match the key name stored in Azure Key Vault

# Get key securely from Azure Key Vault via secret scope
storage_key = dbutils.secrets.get(scope=secret_scope, key=secret_key_name)

# Loop through containers and mount them if not already mounted
for container_name in containers:
    mount_point = f"{mount_base}/{container_name}"
    configs = {
        f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_key
    }

    if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
            source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
            mount_point=mount_point,
            extra_configs=configs
        )
        print(f"✅ Mounted securely: {mount_point}")
    else:
        print(f"⚠️ Already mounted: {mount_point}")


In [0]:
df_raw = spark.read.format("parquet").load("/mnt/adf-silver")

In [0]:
display(dbutils.fs.mounts())

In [0]:
df_csv = spark.read.option("header", True).csv("/mnt/raw-ingest/finance_invoice_data.csv")
df_json = spark.read.option("multiLine", True).json("/mnt/external-ingest/web_form_submissions.json")
